In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
import unidecode
import string
import datetime
import time
from IPython.display import clear_output
import urllib3
import certifi
import matplotlib.pyplot as plt
import seaborn as sns

In [136]:
us_PD = pd.DataFrame()

In [135]:
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

In [156]:
len(states)

50

In [145]:
for s in states:
    print('-'.join(s.lower().split()))

alabama
alaska
arizona
arkansas
california
colorado
connecticut
delaware
florida
georgia
hawaii
idaho
illinois
indiana
iowa
kansas
kentucky
louisiana
maine
maryland
massachusetts
michigan
minnesota
mississippi
missouri
montana
nebraska
nevada
new-hampshire
new-jersey
new-mexico
new-york
north-carolina
north-dakota
ohio
oklahoma
oregon
pennsylvania
rhode-island
south-carolina
south-dakota
tennessee
texas
utah
vermont
virginia
washington
west-virginia
wisconsin
wyoming


In [150]:
us_PD = pd.DataFrame()
for s in states:
    
    # instantiate zults
    zults = {'name':[], 'url':[], 'pop':[], 'officers':[]}

    # instantiate HTTP certificate
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

    # instantiate url
    state = '-'.join(s.lower().split())
    print(state)
    n = 1

    run = True

    while run:
        url = f'https://www.policeone.com/law-enforcement-directory/search/page-{n}/?state={state}&type=police%20departments'

        # start request
        r = http.request('GET', url)

        # check status
        if r.status == 200:
            soup = BeautifulSoup(r.data, 'lxml')
            if len(soup.find_all('a', {'class':'Table-row'})) == 0:
                run = False
                continue
            else:
                for span in soup.find_all('span', {'class':'u-textLink'}):
                    zults['name'].append(span.text.strip())
                for link in soup.find_all('a', {'class':'Table-row'}):
                    zults['url'].append(link['href'])
        n += 1

    # pull linked data
    for l in zults['url']:
        print(l)
        data = {'dd':[],'dt':[]}
        r3 = http.request('GET', f'https://www.policeone.com{l}')
        soup = BeautifulSoup(r3.data, 'lxml')
        for d in soup.find_all('dd', {'class':'DefList-description'}):
            data['dd'].append(d.text.strip())
        for t in soup.find_all('dt', {'class':'DefList-term'}):
            data['dt'].append(t.text.strip())
        d2 = dict(zip(data['dt'], data['dd']))
        try:
            zults['pop'].append(d2['Population Served:'])
        except KeyError:
            zults['pop'].append('missing')
        try:
            zults['officers'].append(d2['Number of Officers:'])
        except KeyError:
            zults['officers'].append('missing')
    
    zult = pd.DataFrame(zults)
    zult['state'] = state
   
    us_PD = pd.concat([us_PD, zult])
    clear_output()

        

In [158]:
for s in states:
    x = ('-'.join(s.lower().split()))
    if x not in us_PD['state'].unique():
        print(x)

rhode-island
vermont


In [157]:
us_PD['state'].unique()

array(['alabama', 'alaska', 'arizona', 'arkansas', 'california',
       'colorado', 'connecticut', 'delaware', 'florida', 'georgia',
       'hawaii', 'idaho', 'illinois', 'indiana', 'iowa', 'kansas',
       'kentucky', 'louisiana', 'maine', 'maryland', 'massachusetts',
       'michigan', 'minnesota', 'mississippi', 'missouri', 'montana',
       'nebraska', 'nevada', 'new-hampshire', 'new-jersey', 'new-mexico',
       'new-york', 'north-carolina', 'north-dakota', 'ohio', 'oklahoma',
       'oregon', 'pennsylvania', 'south-carolina', 'south-dakota',
       'tennessee', 'texas', 'utah', 'virginia', 'washington',
       'west-virginia', 'wisconsin', 'wyoming'], dtype=object)

In [160]:
us_PD.to_csv('police_one_scrape_722.csv')